# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Week 4: From MLP over RNN to Transformer</font>

# <font color="#003660">Notebook 3: Long-Short Term Memory (LSTM) Networks with Embeddings</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... understand the logic behind recurrent neural networks, especially LSTMs. and <br>
        ... are able to train a LSTM with word embeddings as features.
    </font>
</div>
</center>
</p>

# What are Recurrent Neural Networks?

## Simple Recurrent Neural Network (RNN)

A RNN processes sequences by iterating through the sequence elements and maintaining a *state* containing information relative to what it has seen so far. In effect, a RNN layer is a neural network layer with an internal loop, as shown in the figure below.

<br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/rnn1.png"/><br>
<center>Source: Chollet (2021)</center>

The figure below shows a simple RNN unrolled over time. As can be seen from the figure the output of a layer is a combination of

1. its direct data input (`input_t`),
2. the layer's state from the previous timestep (`state_t`), and
3. a bias term (`bo`).

<br><img width=700 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/rnn2.png"/><br>

<center>Source: Chollet (2021)</center>

## Long-Short Term Memory (LSTM) Networks

Compared to a simple RNN layer, a LSTM layer contains one central innovation: A *carry track* that allows to carry over information over time from any previous timestep to the current timestep. Consequently, the output of a layer is a combination of



1. its direct data input (`input_t`),
2. the layer's state from the previous timestep (`state_t`),
3. the input from the carry track (`c_t`), and
4. a bias term (`bo`).

<br><img width=700 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/lstm.png"/><br>
<center>Source: Chollet (2021)</center>

## Comparing RNNs with MLPs

Andrej Karpathy's legendary blog post "The Unreasonable Effectiveness of Recurrent Neural Networks" contains a very informative comparison of RNNs with traditional neural networks: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

# Import packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `numpy` is a library adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `SQLAlchemy`, together with `pymysql`, allows to communicate with SQL databases.
- `getpass` provides function to safely enter passwords.
- `sklearn` is a free software machine learning library for the Python programming language.
- `tensorflow` is an end-to-end open source platform for machine learning, especially deep learning.
- `matplotlib` is a plotting library for the Python programming language and its numerical mathematics extension NumPy


In [ ]:
# Install packages
!pip install pymysql

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import getpass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from sklearn import metrics
import matplotlib.pyplot as plt

Check if we are running on GPU.

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

# Load documents

We load our data from a MySQL database. For security reasons, we don't store the database credentials here; please have a look at Panda to get them.

In [ ]:
# Get credentials
user = input("Username: ")
passwd = getpass.getpass("Password: ")
server = input("Server: ")
db = input("Database: ")

# Create an engine instance (SQLAlchemy)
engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, passwd ,server, db))

# Define SQL query
sql_query = "SELECT * FROM WineDataset"

# Query dataset (pandas)
corpus = pd.read_sql(sql=sql_query, con=engine)

Display `shape` of the data.

In [ ]:
corpus.shape

Split data into three sets: training, validation, and test.

In [ ]:
train_corpus = corpus[corpus["testset"] == 0]
val_corpus = train_corpus.iloc[80000:100000,]
train_corpus = train_corpus.iloc[0:80000,].sample(10000)
test_corpus = corpus[corpus["testset"] == 1]

For each dataset, store features and targets in separate variables

In [ ]:
train_corpus_features = train_corpus[["description"]]
train_corpus_target = train_corpus[["points"]]
val_corpus_features = val_corpus[["description"]]
val_corpus_target = val_corpus[["points"]]
test_corpus_features = test_corpus[["description"]]
test_corpus_target = test_corpus[["points"]]

Create [TensorFlow `Datasets`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) from the Pandas Dataframes. The use of TensorFlow Datasets follows a common pattern:

1.   Create a dataset from raw data (e.g., a Pandas dataframe, a CSV file, multiple text files).
2.   Apply transformations to preprocess the data in the dataset (e.g., vectorize text data).
3. Iterate over the dataset and process its elements. Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

Here, we use the `from_tensor_slices` constructor to create datasets from dataframes.

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((tf.cast(train_corpus_features.values, tf.string),
                                               tf.cast(train_corpus_target.values, tf.int32)))

val_ds = tf.data.Dataset.from_tensor_slices((tf.cast(val_corpus_features.values, tf.string),
                                             tf.cast(val_corpus_target.values, tf.int32)))

test_ds = tf.data.Dataset.from_tensor_slices((tf.cast(test_corpus_features.values, tf.string),
                                              tf.cast(test_corpus_target.values, tf.int32)))

Display some stats and examples from the created datasets.

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("===")
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

# Vectorize documents

We will now use [TensorFlow's `TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) function to transform raw texts into numerical vectors. Again, we map unique words to integers.

In [ ]:
max_tokens = 10000
max_length = 100

text_vectorization = TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length
)

Some apects of the `TextVectorization` function (e.g., the size and contents of the vocabulary) have to be fit using training data, which can be done with the `adapt` function.

In [ ]:
train_ds_features_only = train_ds.map(lambda x, y: x)
text_vectorization.adapt(train_ds_features_only)

Show the vocabulary that our vectorizer knows after being fit to the training data.

In [ ]:
text_vectorization.get_vocabulary()[0:10]

Next, we apply our `text_vectorization` function to all three datasets. This corresponds to step 2 mentioned above.

In [ ]:
vectorized_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

vectorized_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

vectorized_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

Show results.

In [ ]:
for inputs, targets in vectorized_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("===")
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

# Train model

We are now ready to specify a neural network and feed it with the vectroized datasets. For convenience, we define a custome function `get_model` which defines the network architecture, creates a model from it, and compiles this model (by defining, e.g., an otpimizer and loss function).

Instead of averaging or flattening the outputs of the embedding layer, a LSTM layer can directly process its 2D output (i.e., it takes a sequence of vectors as input instead of a single vector).

In the example below we stack two bi-directional LSTM layers on top of each other.

In [ ]:
def get_model(max_tokens=10000):
    inputs = keras.Input(shape=(max_length,), dtype="int64")
    embedded = layers.Embedding(input_dim=max_tokens, output_dim=200, mask_zero=True)(inputs)
    hidden1 = layers.Bidirectional(layers.LSTM(32, return_sequences = True))(embedded)
    hidden2 = layers.Bidirectional(layers.LSTM(32))(hidden1)
    outputs = layers.Dense(1, activation = "linear")(hidden2)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer = tf.optimizers.Adam(),
                  loss = "mean_absolute_error",
                  metrics = ["mean_absolute_error"])
    return model

Instantiate model and show it's architecture.

In [ ]:
model = get_model(max_tokens)
model.summary()

Fit model on training data and save best model to disk.

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("lstm.keras", save_best_only=True)]

history = model.fit(vectorized_train_ds.cache(),
          validation_data = vectorized_val_ds.cache(),
          epochs = 3,
          batch_size = 64,
          callbacks = callbacks)

Plot the learning process.

In [ ]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Model accuracy')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Make predictions

Load best model from training phase.

In [ ]:
model = keras.models.load_model("lstm.keras")

Make predictions on test set.

In [ ]:
preds = model.predict(vectorized_test_ds)

Calculate accuracy metrics.

In [ ]:
print(metrics.mean_absolute_error(test_corpus_target, preds))